Learning to Learn - John Tomlinson, Ahan Maurya, Dr. Jing Wang

First we need to import the modules we created for this notebook

In [16]:
import meta_network
import original_network
import numpy as np
import matplotlib.pyplot as plt

In [17]:
# Reload to pick up on-disk changes to original_network
import importlib
importlib.reload(original_network)

<module 'original_network' from 'c:\\Users\\bobtt\\OneDrive\\Desktop\\Documents\\JackCS\\learning-to-learn\\cartpole-implementation\\original_network.py'>

We will start by initializing both of our networks

In [18]:
cartpole_network, cartpole_target_network,memory, optimizer, device, env = original_network.build_model()
meta_network, meta_memory = meta_network.build_model(1,1)

Now lets build a function to graph our reward over time

In [19]:
def plot_rewards(reward_list):
    plt.plot(reward_list)
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.title('Reward over Time')
    plt.show()

Lets make a training loop to get our baseline performance for our model wihtout the meta learning

In [20]:
NUM_EPISODES = 300
TARGET_UPDATE = 10

In [13]:
def training_loop(episodes):
    total_reward_list = []
    epsilon = 0.1  # Exploration rate
    for episode in range(episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = original_network.select_action(state, cartpole_network, device, epsilon, env)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            memory.push(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

        original_network.optimize_model(cartpole_network, cartpole_target_network, memory, optimizer, device)

        if episode % TARGET_UPDATE == 0:
            cartpole_target_network.load_state_dict(cartpole_network.state_dict())

        print(f"Episode {episode+1}/{episodes}, Total Reward: {total_reward}")
        total_reward_list.append(total_reward)
    return total_reward_list

In [14]:
plot_rewards(training_loop(NUM_EPISODES))

Episode 1/300, Total Reward: 9.0


NameError: name 'env' is not defined